In [2]:
from langchain.graphs import Neo4jGraph
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Reason@@"

graph = Neo4jGraph()

In [3]:
import requests
import pandas as pd

# Define the URL and parameters
url = "https://datasets-server.huggingface.co/rows"
params = {
    "dataset": "FreedomIntelligence/medical-o1-reasoning-SFT",
    "config": "en",              # this matches the viewer page path
    "split": "train",
    "offset": 0,
    "length": 100
}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Convert the JSON data to a Pandas DataFrame
    rows = data.get('rows', [])
    df = pd.DataFrame([row['row'] for row in rows])

In [5]:
df.columns

Index(['Question', 'Complex_CoT', 'Response'], dtype='object')

In [6]:
import pandas as pd
from langchain.docstore.document import Document


# Define the function to format each row
def format_row(row):
    return (
        f"Question: {row['Question']}\n"
        f"Response Reasoning: {row['Complex_CoT']}\n"
    ).lower()

# Apply the function to each row and create a new column with the formatted text
df['formatted_text'] = df.apply(format_row, axis=1)

#Convert the formatted text into a list of Document objects
documents = []
for text in df['formatted_text'].head(50).tolist():
    document = Document(page_content=text)
    documents.append(document)


In [7]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(documents)

print("Documents after splitting:")
for doc in documents:
    print(doc.page_content)

Documents after splitting:
question: a 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and q-tip test. based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?
response reasoning: okay, let's think about this step by step. there's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. this sounds a lot like stress urinary incontinence to me. now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. this likely means her bladder's ability to hold urine is fine when she isn't under physical stress. hmm, that's a clue that we're dealing with something related to pressure rather than a bladder muscle problem. 

the fact that she underwent a q-tip test is

In [8]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi")
llm.invoke("Tell me a joke")

C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_49472\3729597464.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi")


" Sure, here's one for you! Why don't scientists trust atoms? \n\nBecause they make up everything! \n\nI hope that got a smile on your face! Do you have any more questions or would you like another joke?\n"

In [9]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.schema import Document

# Khởi tạo LLM transformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Lấy subset tài liệu (bạn có thể thay đổi số lượng nếu muốn)
subset_documents = documents[:30]

# Kiểm tra định dạng tài liệu đầu vào
for i, doc in enumerate(subset_documents):
    if not isinstance(doc, Document):
        raise ValueError(f"Tài liệu số {i} không phải là một instance của langchain.schema.Document")
    if not isinstance(doc.page_content, str):
        raise TypeError(f"page_content của tài liệu {i} không phải chuỗi. Loại: {type(doc.page_content)}")

# Chuyển từng document một để dễ kiểm soát lỗi
graph_documents = []
for i, doc in enumerate(subset_documents):
    try:
        gdoc = llm_transformer.convert_to_graph_documents([doc])[0]
        
        # Làm sạch relationships
        for rel in gdoc.relationships:
            if isinstance(rel["head"], dict):
                print(f"Document {i}: head là dict, chuyển thành chuỗi.")
                rel["head"] = str(rel["head"])
            if isinstance(rel["tail"], dict):
                print(f"Document {i}: tail là dict, chuyển thành chuỗi.")
                rel["tail"] = str(rel["tail"])
            if not rel["head"] or not rel["tail"]:
                print(f"Warning: Relationship rỗng trong document {i}.")
        
        graph_documents.append(gdoc)
    
    except Exception as e:
        print(f"Lỗi khi xử lý document {i}: {e}")

# Đẩy dữ liệu lên Neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=False,
    include_source=True
)


Lỗi khi xử lý document 8: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 9: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 10: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 13: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 16: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 20: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 21: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 27: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 29: 'Relationship' object is not subscriptable


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5" # hoặc "cpu"
)

vector = embeddings.embed_query("Test embedding")
print(vector[:10])  # In thử 10 phần tử đầu


c:\Users\COHOTECH\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


[0.02927350625395775, -0.045802462846040726, -0.015977395698428154, 0.06701088696718216, 0.059911761432886124, 0.05131841078400612, 0.03220994397997856, -0.022907845675945282, 0.03211498633027077, -0.032036054879426956]


In [12]:

from langchain_community.vectorstores import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

In [14]:
df.Question.unique()

array(['A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?',
       'A 45-year-old man with a history of alcohol use, who has been abstinent for the past 10 years, presents with sudden onset dysarthria, shuffling gait, and intention tremors. Given this clinical presentation and history, what is the most likely diagnosis?',
       'A 45-year-old man presents with symptoms including a wide-based gait, a blank facial expression, hallucinations, memory issues, a resting tremor that resolves with movement, and bradykinesia. Based on these clinical findings, what is most likely to be observed in the histological specimen of his brain?',
       'A patient with psoriasis was treated with systemic steroids, and upon discontinuation of the treatment, 

In [15]:
import gradio as gr

# Define the function for querying patient details
def query_patient_details(query):
    try:
        result = qa_chain({"query": query})
        return result["result"]
    except Exception as e:
        return f"Error: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=query_patient_details,        # Function to call
    inputs=gr.Textbox(label="Enter your question"),  # Input textbox
    outputs=gr.Textbox(label="Answer")   # Output textbox
)

# Launch the interface
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://d51abc90f312dae2c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_49472\3595798144.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})
